# RIFRAF clonal accuracy

This is a [Jupyter](http://jupyter.org/) notebook that demonstrates how to use RIFRAF on clonal sequence data. This notebook is a work in progress.

The method is implemented as a Julia package: [Rifraf.jl](https://github.com/MurrellGroup/Rifraf.jl). The data is available on FigShare: [https://doi.org/10.6084/m9.figshare.5643247](https://doi.org/10.6084/m9.figshare.5643247).

A full description of the method has been submitted for publication. A link to the journal article will be added here when it is available.

## Setup
 
Before running for the first time, do the following:

- Install `Rifraf.jl`.
- Download and unzip the data from FigShare (or supply your own data)
- Update the paths to the data in the code.

In [ ]:
using Rifraf

# update these paths
READ_FILE = "../nl43_data/C01.NL43.fastq";
REFERENCE_FILE = "../nl43_data/references.fasta";

# set these parameters
const ERROR_RANGE = (1e-3, 1e-2)
const PHRED_CAP = Int8(30)
const LENGTH_CUTOFF = 25

const REF_SCORES = Scores(ErrorModel(9.0, 1e-5, 1e-5, 0.5, 0.5))
const SEQ_SCORES = Scores(ErrorModel(1.0, 5.0, 5.0, 0.0, 0.0));

## Utility code

The following code is used for reading and processing the data.

In [2]:
include("./clonal_code.jl");

## Run experiments

In [4]:
function run_experiments()
    # read data
    references = Rifraf.read_fasta_records(REFERENCE_FILE)
    sequences, phreds, names = Rifraf.read_fastq(READ_FILE, seqtype=DNASeq)
    
    # use first reference as both template as reference, just for demonstration purposes
    refseq = DNASeq(sequence(references[1]))
    template = DNASeq(sequence(references[1]))
    
    # sample a set of reads
    mean_errors = Float64[mean(Rifraf.phred_to_p(phred)) for phred in phreds]
    cand_indices = valid_read_indices(ERROR_RANGE, sequences, phreds, mean_errors; length_cutoff=LENGTH_CUTOFF)
    sampled = sample_reads(10, cand_indices, sequences, phreds, names, mean_errors)
    (sampled_sequences, sampled_phreds, sampled_names, sampled_indices) = sampled
    
    # trim ends against the first reference
    trimmed_sequences, trimmed_phreds = trim_reads(sampled_sequences, sampled_phreds, refseq)
    
    # run RIFRAF with capped phreds and the first reference sequence
    params = RifrafParams(scores=SEQ_SCORES, ref_scores=REF_SCORES)
    capped_phreds = Vector{Int8}[Rifraf.cap_phreds(p, PHRED_CAP) for p in trimmed_phreds]

    tic()
    result = rifraf(trimmed_sequences, capped_phreds; reference=refseq, params=params)
    elapsed = toq()
    
    println("converged: $(result.state.converged)")
    println("execution time: $(round(elapsed, 2)) seconds")
    println("found true sequence: $(result.consensus == template)")
end

run_experiments()

converged: true
execution time: 3.6 seconds
found true sequence: true
